# Web APIs

An Application Programming Interface (API) is a contract between two
systems: if the client sends a request in a specific format it will
always receive a response from the service in a specific format or
trigger a defined action. APIs are not limited to the internet but in
many cases when people refer to an API they are referring to RESTful web
APIs with communication between a client and a server over the web. This
chapter shows you how to interact with web APIs in Python using the
[Requests](https://python-requests.org/) library.

## HTTP in a nutshell

The protocol of the web is [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). This is used for serving up traditional web pages and also web services exposed via web APIs.

HTTP is a simple client-server protocol:

![HTTP client-server protocol](extras/http_client_server_architecture.png)

Your web browser is an HTTP client. So is the Python `requests` library.

HTTP is a simple language (protocol). Its most important verb is GET. Most pages display in your browser from GET requests. We can issue a GET request from Python using `requests` as follows:

In [2]:
import requests

url = "http://www.google.com"
response = requests.get(url)
print(response.text[:200])

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en-AU"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/


What we see above is the first 200 characters returned by the HTTP server at `google.com`.

Making a `GET` request from `www.google.com` only returns the web content necessary for rendering the google home page in our browser.  Typically, however, we are using APIs in order to acquire data.  To see an example of this, we shall use the public World Bank Open Data API (documentation available at https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information).

We're going to use this API to retrieve a list of all the income levels with which the World Bank classifies countries.  The URL we need to use is `https://api.worldbank.org/v2/incomeLevels`.  

Note the `v2` after `api.worldbank.org`: this signifies that we're accessing version two of the API and also lets us know that this (as well as any decent API) is concerned with stability such that we can be confident that the behaviour of this API, so long as we use `v2`, will not change or break our code.

Retrieving our desired data works much the same as above.

In [2]:
url = 'https://api.worldbank.org/v2/incomeLevels'
response = requests.get(url)
print(response.text)

ï»¿<?xml version="1.0" encoding="utf-8"?>
<wb:IncomeLevels page="1" pages="1" per_page="50" total="7" xmlns:wb="http://www.worldbank.org">
  <wb:incomeLevel id="HIC" iso2code="XD">High income</wb:incomeLevel>
  <wb:incomeLevel id="INX" iso2code="XY">Not classified</wb:incomeLevel>
  <wb:incomeLevel id="LIC" iso2code="XM">Low income</wb:incomeLevel>
  <wb:incomeLevel id="LMC" iso2code="XN">Lower middle income</wb:incomeLevel>
  <wb:incomeLevel id="LMY" iso2code="XO">Low &amp; middle income</wb:incomeLevel>
  <wb:incomeLevel id="MIC" iso2code="XP">Middle income</wb:incomeLevel>
  <wb:incomeLevel id="UMC" iso2code="XT">Upper middle income</wb:incomeLevel>
</wb:IncomeLevels>


From the first line you can see that the data we have gotten back is in `XML` format which has become unpopular in recent years and often poses data parsing problems that are unnecessarily complex.

The more popular and user-friendly format we want to use is `JSON`, which turns out to be expressed in a way very similar to the way python creates lists and dictionaries!

To get data back in `JSON` format we will have to add `format=json` to our `URL` as follows.

In [3]:
url = 'https://api.worldbank.org/v2/incomeLevels?format=json'
response = requests.get(url)
print(response.text)

[{"page":"1","pages":"1","per_page":"50","total":"7"},[{"id":"HIC","iso2code":"XD","value":"High income"},{"id":"INX","iso2code":"XY","value":"Not classified"},{"id":"LIC","iso2code":"XM","value":"Low income"},{"id":"LMC","iso2code":"XN","value":"Lower middle income"},{"id":"LMY","iso2code":"XO","value":"Low & middle income"},{"id":"MIC","iso2code":"XP","value":"Middle income"},{"id":"UMC","iso2code":"XT","value":"Upper middle income"}]]


Immediate you should be able to see the resemblance between this format and our python lists and dictionaries.  Hopefully by comparing with the above `XML` output you can confirm that the same raw data is contained in both outputs.

But, so far our data is just a string, not an actual list or dictionary that we can easily analyse with python.  To parse the `JSON` the raw data into a python object, instead of using `response.text`, we must use the built-in function `json()`, which will return a python list or dictionary as needed.

In [4]:
income_levels = response.json()
print(income_levels[1][0])

{'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'}


In [5]:
income_level_ids = []

for il in income_levels[1]:
    income_level_ids.append(il['id'])
print(income_level_ids)    

['HIC', 'INX', 'LIC', 'LMC', 'LMY', 'MIC', 'UMC']


### Exercise

The World Bank collects data from many sources. The API allows us to retrieve a complete list of all the sources used.  To access this the URL used above must have `incomeLevels` replaced by `sources`.  Adapt the above code to obtain a list of the World Bank's sources.  Don't forget to retrieve the data in `JSON` format.

## URLs and Parameters

You will notice that the income levels URL contained a question mark and that it was after this question mark that we specified the response format as `JSON`.  This is a common pattern with APIs.  

Everything before the question mark is a `URL` or address or *endpoint* and serves to define a specific API or target from which one can make requests.  Everything after the question mark provides additional `parameters` or arguments to the API to further specify how the request is going to work.  In the above example we used the `format` parameter to specify the format of our response data.  

The parameters that an API accepts and processes is arbitrary and must be observed from the API's documentation.  For instance, a number of additional parameters, many of which relate to specifying time periods, that are available for the World Bank API are documented at https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures.

The recommended way of handling parameters with the `requests` library is to express them as a dictionary and pass this dictionary to the request as the `params` argument.  This avoids the error-prone and long `URLs` that parameters can cause.

In [6]:
url = 'https://api.worldbank.org/v2/incomeLevels'
params = {'format': 'json'}

response = requests.get(url, params=params)
print(response.json())

[{'page': '1', 'pages': '1', 'per_page': '50', 'total': '7'}, [{'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'}, {'id': 'INX', 'iso2code': 'XY', 'value': 'Not classified'}, {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'}, {'id': 'LMC', 'iso2code': 'XN', 'value': 'Lower middle income'}, {'id': 'LMY', 'iso2code': 'XO', 'value': 'Low & middle income'}, {'id': 'MIC', 'iso2code': 'XP', 'value': 'Middle income'}, {'id': 'UMC', 'iso2code': 'XT', 'value': 'Upper middle income'}]]


### Countries and Indicators

The two main endpoints provided by the World Bank API for retrieving data are `/countries` (i.e. `https://api.worldbank.org/v2/countries`) and `/indicators` (which specify the metric of interest).  Generally these two are combined to obtain the data one desires.

The remaining endpoints are primarily for retrieving secondary, descriptive or metadata.  They are `/incomeLevels`, `/lendingTypes`, `/sources` and `/topics`.

#### Listing Sources, Indicators and Countries

The World Bank's indicators are obtained from a limited set of sources typically with a general focus.  Searching for an indicator of interest can therefore be done by retrieving the available sources and then the indicators under a particular source.

In [7]:
# Getting list of sources

resp = requests.get('https://api.worldbank.org/v2/sources', 
                    params={'format':'json', 'per_page':'100'})

In [8]:
resp.json()[1][:2]

[{'id': '1',
  'lastupdated': '2019-10-23',
  'name': 'Doing Business',
  'code': 'DBS',
  'description': '',
  'url': '',
  'dataavailability': 'Y',
  'metadataavailability': 'Y',
  'concepts': '3'},
 {'id': '2',
  'lastupdated': '2020-07-01',
  'name': 'World Development Indicators',
  'code': 'WDI',
  'description': '',
  'url': '',
  'dataavailability': 'Y',
  'metadataavailability': 'Y',
  'concepts': '3'}]

Sometimes a function like that below can be useful for quickly isolating the relevant data from the long and rich lists APIs return

In [9]:
def get_fields(data, fields):
    '''Retrieve specified fields for each entry in data'''
    data_list = []
    for entry in data:
        entry_list = []
        for f in fields:
            entry_list.append(entry[f])
        data_list.append(entry_list)
        
    return data_list

In [10]:
sources_clean = get_fields(resp.json()[1], ['id', 'code', 'name'])

print(sources_clean[:3])

[['1', 'DBS', 'Doing Business'], ['2', 'WDI', 'World Development Indicators'], ['3', 'WGI', 'Worldwide Governance Indicators']]


### Exercise

Using the indicators API (URL: `https://api.worldbank.org/v2/indicators`) and the parameter `source` along with the `id` of the source we're interested in, we can obtain a list of all the indicators originating from the specified source.

Retrieve a list of available sources.  Select a source of interest, retrieve the indicators available from it and select one of those of interest.  Don't forget to use the parameters `per_page` and/or `page` to obtain all possible results if necessary.

**Extension**

Using the `/countries` endpoint (i.e. `https://api.worldbank.org/v2/countries`), write a function that determines what the `id`/`isocode` or the `iso2code` for a particular country (such as 'Aruba').

Using either string methods or regular expression, can you write a function that allows you to easily search the names of the various entries the API gives back to you for a particular word like *population*?

### Getting Country Data

To retrieve data for a specific indicator either for all countries or a specific country requires combining the `/indicators` and `/countries` endpoints, along with the specific country and indicator we're interested in, into a single `URL`.

For example, to retrieve data for the population indicator `SP.POP.TOTL`, found under the source `World Development Indicators`, for the United States or `USA`, would require the following `URL`:
`https://api.worldbank.org/v2/countries/USA/indicators/SP.POP.TOTL`

This pattern of using `URL`s instead of parameters to query an API is not uncommon.  The World Bank have combined both parameters and complex URLs in the design of their API.  Fortunately `python's` string functionality is very useful for such tasks, especially `f-strings`.

In [11]:
url = 'https://api.worldbank.org/v2'
country = 'USA'
indicator = 'SP.POP.TOTL'
url = f'{url}/countries/{country}/indicators/{indicator}'

resp = requests.get(url, params={'format':'json'})

pop_data = resp.json()

print(pop_data[1][:3])

[{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'US', 'value': 'United States'}, 'countryiso3code': 'USA', 'date': '2019', 'value': 328239523, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'US', 'value': 'United States'}, 'countryiso3code': 'USA', 'date': '2018', 'value': 326687501, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'}, 'country': {'id': 'US', 'value': 'United States'}, 'countryiso3code': 'USA', 'date': '2017', 'value': 324985539, 'unit': '', 'obs_status': '', 'decimal': 0}]


### Exercise

Lets write a function that returns the GDP (or your preferred indicator) for any given country.

A good GDP indicator is the "*GDP (constant 2010 US$)*" (id: `NY.GDP.MKTP.KD`).

Adapt the code above into a function that should operate like the following:

In [ ]:
USA_gdp = get_gdp('USA')

print(USA_gdp)  # 18318722096911

Don't forget that countries need to be specified in either 3 or 2 letter `iso` format (e.g. `BR` or `BRA` for Brazil).

**Extensions**

Notice that the data returned lists historical yearly data for GDP.  Read the [API documentation](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures) to determine how to use parameters to specify a date (*hint*: `date=2000`) or to limit the number of values returned (*hint*: `MRV=1`).

The API is capable of accepting multiple countries in the `URL` so long as they are separated by a semi-colon (`;`).  Modify your function to be able to take a list of countries as an argument.  You will need to format the multiple countries appropriately within the function for the `URL`.  You will also need to alter the output of your function to accommodate the multiple values you will have.

The API can also accept multiple indicators so long as they too are separated by semi-colons like the countries.  When doing so, however, you will often need to also provide the source of the indicators as a parameter (the API can get confused otherwise).  Presuming all indicators come from source `id: 2`, can you make a function that accepts and manages `incicators`, `countries` and `dates` arguments?



### Data Analysis with Pandas

Often the best way to get a handle on large and rich data structures like those provided by APIs is to find a way to get your data into a `pandas` dataframe and use all of the tools provided by that library.

Let's try to find the correlation between two indicators.

To begin, we must first retrieve data efficiently.  From the exercises above, let us settle on a utility function for retrieving data like so:

In [12]:
def get_indicator_data(indicators, date='2019', countries='all', source='2', per_page=1000):
    '''Retrieve data for specified countries and indicators'''
    
    url = 'https://api.worldbank.org/v2'
    
    if isinstance(countries, list):
        countries = ';'.join(countries)

    if isinstance(indicators, list):
        indicators = ';'.join(indicators)
        
    url = f'{url}/countries/{countries}/indicators/{indicators}'
    
    parameters = {
        'format': 'json',
        'date': date,
        'source': source, 
        'per_page': per_page
    }
    
    resp = requests.get(url, params=parameters)
    resp_data = resp.json()
    
    return resp_data

To give it a spin, let's get data for the "*GDP per capita (constant 2010 US$)*" indicator (id: `NY.GDP.PCAP.KD`) and the "*School enrollment, primary (% gross)*" (id: `SE.PRM.ENRR`), for the United States and for the year 2012 as not all data is reliably updated.

In [13]:
data = get_indicator_data(['SE.PRM.ENRR', 'NY.GDP.PCAP.KD'], countries='usa', date='2012')
print(data)

[{'page': 1, 'pages': 1, 'per_page': 1000, 'total': 2, 'sourceid': None, 'lastupdated': '2020-07-01'}, [{'indicator': {'id': 'SE.PRM.ENRR', 'value': 'School enrollment, primary (% gross)'}, 'country': {'id': 'US', 'value': 'United States'}, 'countryiso3code': 'USA', 'date': '2012', 'value': 99.16532, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2010 US$)'}, 'country': {'id': 'US', 'value': 'United States'}, 'countryiso3code': 'USA', 'date': '2012', 'value': 49603.2534736283, 'unit': '', 'obs_status': '', 'decimal': 1}]]


Now, to create a flat table of data from this, we're going to have to flatten the nested dictionaries under `indicator` and `country`.  A function like that below which loops through each entry and creates a new simpler dictionary will do the trick.

In [14]:
def clean_wbdata(data):
    
    new_data = []
    for value in data:
        new_value = {
            'indicator': value['indicator']['value'],
            'country': value['country']['value'],
            'date': value['date'],
            'value': value['value']
        }
        
        new_data.append(new_value)
        
    return new_data

In [15]:
clean_data = clean_wbdata(data[1])

print(clean_data)

[{'indicator': 'School enrollment, primary (% gross)', 'country': 'United States', 'date': '2012', 'value': 99.16532}, {'indicator': 'GDP per capita (constant 2010 US$)', 'country': 'United States', 'date': '2012', 'value': 49603.2534736283}]


Now that the structure of the data is flat and square, `pandas` is capable of creating a `DataFrame` from it.

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(clean_data)
df

indicator        country  date         value
0  School enrollment, primary (% gross)  United States  2012     99.165320
1    GDP per capita (constant 2010 US$)  United States  2012  49603.253474

Given the structure of this data and our purpose, transforming it from `long-form` to `wide-form` will be helpful.  The `df.pivot` function performs this task.

In [18]:
df = df.pivot(index='country', columns='indicator', values='value')
df

indicator      GDP per capita (constant 2010 US$)  \
country                                             
United States                        49603.253474   

indicator      School enrollment, primary (% gross)  
country                                              
United States                              99.16532

### Exercise

Retrieve data as done above but for all countries.  You should end up with a much larger dataframe than the one above.  Find the correlation between the two indicators.

Perform the same analysis but with "*Mortality rate, infant (per 1,000 live births)*" (id: `SP.DYN.IMRT.IN`) indicator rather than previous schooling indicator (id: `SE.PRM.ENRR`).  What's the correlation between infant mortality and GDP per capita?


**Extensions**

Can you wrap your workflow up into a function that returns a dataframe from just the indicator, country and data arguments like the function above?  You could use this to find the correlation between other indicators of interest

Each country is classified as being of a particular income level.  This data is available under the `/countries` endpoint and could be joined to our indicator dataframes using the `join` or `merge` method.  From this, the distributions of the indicators could be compared between the income levels, especially through appropriate plots.

## POST

Above we saw GET requests. Although this is perhaps the most common HTTP verb in web APIs, another important one is POST.

POST corresponds to submission of data, traditionally via a HTML form. A POST request is expected to change the state on the server (whereas GET does not).

<img src="https://secureservercdn.net/160.153.137.163/84g.4be.myftpupload.com/wp-content/uploads/2019/06/POST-vs-GET.jpg" width=400>

### GET vs POST in detail
<table>
<tbody>
<tr>
<td><strong>Parameters</strong></td>
<td><strong>POST</strong></td>
<td><strong>GET</strong></td>
</tr>
<tr>
<td>Data Parameters</td>
<td>It does not store and includes the data parameters in the URL or anywhere on the client system</td>
<td>It includes the data parameters in the URL which can store in the browser history and the security of the client can be compromised.</td>
</tr>
<tr>
<td>Bookmark</td>
<td>As the data does not include in URL so there is no concept of Bookmark</td>
<td>The get request data can be Bookmarked</td>
</tr>
<tr>
<td>Reload the page</td>
<td>The data will be resubmitted.</td>
<td>The page will be re-executed but not re-submitted, it does not send any request to the host because it is saved in the cache memory.</td>
</tr>
<tr>
<td>Security</td>
<td>POST provide security over the data</td>
<td>It does not provide any security</td>
</tr>
<tr>
<td>Cache</td>
<td>No cache</td>
<td>There can be caches</td>
</tr>
<tr>
<td>Data length</td>
<td>There is no limit for data length</td>
<td>It has a limit of 2048 characters.</td>
</tr>
<tr>
<td>Data type</td>
<td>All data types are allowed</td>
<td>Only accept ASCII characters</td>
</tr>
<tr>
<td>Data visibility</td>
<td>Data is not visible to everyone</td>
<td>Data is visible to everyone</td>
</tr>
<tr>
<td>Use</td>
<td>We use the POST method with sensitive data like password</td>
<td>GET used with insensitive data.</td>
</tr>
</tbody>
</table>